In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


import os
from pathlib import Path



In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# St. Louis Crime
`civicknowledge.com-stl_crime-1.1.1` Last Update: 2021-03-15T01:36:41

__


This data package extends and updates the [St Louis crime data scraping package
from OpenSTL.](https://github.com/OpenSTL/stl-crime-data). The St. Louis
Metropolitan Police Department [publishes monthly crime incident
files](http://www.slmpd.org/Crimereports.shtml), but they are very difficult to
download and use. The OpenSTL package has Jupyter notebooks for shtmlqcraping
and cleaning the data, but the included data has not been updated in 5 years.

This package retains the original scraper but improces the cleaning process.
See the Clean.ipynb notebok for details of the cleaning process.

## Building the Package

THe metatab package builds from the data in the data directory; the scraper and cleaning notebooks must be run first.


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [st_louis_crime](notevooks/Clean.ipynb)**. Crime Incidents in St Louis

## References
<ul><li> <strong>ucr_codes</string>, <em>data/UCRCodes.csv</em>. </li><li> <strong>neighborhoods</string>, <em>data/neighborhood_lookup.csv</em>. </li><ul>

In [3]:
def duplicated_varnames(df):
    """Return a dict of all variable names that 
    are duplicated in a given dataframe."""
    repeat_dict = {}
    var_list = list(df) # list of varnames as strings
    for varname in var_list:
        # make a list of all instances of that varname
        test_list = [v for v in var_list if v == varname] 
        # if more than one instance, report duplications in repeat_dict
        if len(test_list) > 1: 
            repeat_dict[varname] = len(test_list)
    return repeat_dict


# Column Map
col_map = {
    'AdministrativeAdjustmentIndicator': 'administrativeadjustmentindicator',
 'Beat': 'beat',
 'CAD-Address': 'cad_address',
 'CAD-Street': 'cad_street',
 'CADAddress': 'cad_address',
 'CADStreet': 'cad_street',
 'Coded Month': 'coded_month',
 'CodedMonth': 'coded_month',
 'Complaint': 'complaint',
 'Count': 'count',
 'Crime': 'crime',
 'Date Crime Coded': 'date_coded',
 'Date Occur': 'date_occured',
 'DateOccur': 'date_occured',
 'DateOccured': 'date_occured',
 'Description': 'description',
 'District': 'district',
 'FileName': 'filename',
 'Flag Cleanup': 'flag_cleanup',
 'Flag-Administrative': 'flag_admin',
 'Flag-Crime': 'flag_crime',
 'Flag-Unfounded': 'flag_unfounded',
 'FlagAdministrative': 'flag_admin',
 'FlagCleanup': 'flag_cleanup',
 'FlagCrime': 'flag_crime',
 'FlagUnfounded': 'flag_unfounded',
 'ILEADS-Address': 'ileads_address',
 'ILEADS-Street': 'ileads_street',
 'ILEADSAddress': 'ileads_address',
 'ILEADSStreet': 'ileads_street',
 'ILeads Add': 'ileads_address',
 'ILeads Approve': 'ileads_approve',
 'ILeads Asg': 'ileads_asg',
 'ILeads Type': 'ileads_type',
 'Location Comment': 'location_comment',
 'Location Name': 'location_name',
 'LocationComment': 'location_comment',
 'LocationName': 'location_name',
 'MonthReportedtoMSHP': 'monthreportedtomshp',
 'Neighborhood': 'neighborhood',
 'NewCrimeIndicator': 'newcrimeindicator',
 'UnfoundedCrimeIndicator': 'unfoundedcrimeindicator',
 'X-Coord': 'x',
 'XCoord': 'x',
 'Y-Coord': 'y',
 'YCoord': 'y',
 'Ycoord': 'y',
 'ï»¿Complaint': 'i_complaint'}

# Tweak col map vaules
d = {}
for k, v in col_map.items():
    v = v.lower()
    v = v.replace('-','_')
    v = v.replace(' ','_')
  
    d[k] = v
    
#d # Uncomment to replace the col map. 


In [4]:

df = None
errd_frames = []

p = Path('../cache')

for f in p.glob('*'):
    df_ = pd.read_csv(f, encoding = 'latin1').rename(columns=col_map)
    
    # Using an incremental approach because one of the files cases an error in concat
    if df is None:
        df = df_
    else:
        try:
            df = pd.concat([df, df_])
        except AttributeError as e:
            # at least one file has duplicated columns
            # So try valiantly to fix it
            errd_frames.append(df_)
    
            df = pd.concat([df,df_.loc[:,~df_.columns.duplicated()]])
            print('Error', f, e)
    

Error ../cache/May2017.CSV 'NoneType' object has no attribute 'is_extension'


In [5]:
df.head()

,monthreportedtomshp,date_occured,newcrimeindicator,unfoundedcrimeindicator,administrativeadjustmentindicator,count,crime,district,description,ileads_address,...,flag_crime,flag_unfounded,flag_admin,flag_cleanup,i_complaint,ileads_approve,beat,ileads_asg,ileads_type,date_coded
0,2010-04,01/01/2003 14:15,Y,,,1,115400,1,STLG BY DECEIT/IDENTITY THEFT REPORT,5115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-04,01/01/2009 12:00,Y,,,1,121000,9,EMBEZZLEMENT-VALUE OVER $150,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-04,01/01/2009 12:00,Y,,,1,121000,9,EMBEZZLEMENT-VALUE OVER $150,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04,01/01/2010 00:01,Y,,,1,21000,2,RAPE -- FORCIBLE,5900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04,01/01/2010 12:00,Y,,,1,52312,8,BURGLARY-BUSINESS/UNK TIME/FORC ENT/UNOCCUPIED,4902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Load in neighborhood lookup table and join it with the data frame

In [6]:
# Load in neighborhood lookup table
nbhds = pkg.reference('neighborhoods').dataframe()

In [7]:
df = pd.merge(df, nbhds, left_on = 'neighborhood', right_on = 'Neighborhood Number', how = 'left')
df.drop('Neighborhood Number', axis = 1, inplace = True) # Drop duplicate nbhd number column

## Here we're trying to align the crime codes in the Crime field to the Uniform Crime Reporting codes.
First we need to pad out the Crime column to be six digits, since the leading zero was trimmed in the .csv files

*Need to figure out the numbering system for these crimes.  See UCR, NIBRS, NCIC crime codes and try to make sense of it.*

In [8]:
df['date_occured'] = pd.to_datetime(df.date_occured)

df['crime'] = df['crime'].astype(str)
df['crime'] = df['crime'].apply(lambda x: x.zfill(6))
df['ucr_code'] = df['crime'].apply(lambda x: x[:2])

#Bring in UCR CSV file here and merge df2 and it together on 'ShortenedCrime' column

ucr_codes = pkg.reference('ucr_codes').dataframe()
ucr_codes['UCRCode'] = ucr_codes['UCRCode'].astype(str).apply(lambda x: x.zfill(2))

df = pd.merge(df, ucr_codes, left_on='ucr_code', right_on='UCRCode', how = 'left')\
    .drop('UCRCode', axis = 1)\
    .rename(columns={'UCRType':'ucr_type','UCRCrime':'ucr_description'})
df.head()

,monthreportedtomshp,date_occured,newcrimeindicator,unfoundedcrimeindicator,administrativeadjustmentindicator,count,crime,district,description,ileads_address,...,beat,ileads_asg,ileads_type,date_coded,Neighborhood Name,Neighborhood Primary District,Neighborhood Addl District,ucr_code,ucr_type,ucr_description
0,2010-04,2003-01-01 14:15:00,Y,,,1,115400,1,STLG BY DECEIT/IDENTITY THEFT REPORT,5115,...,NaN,NaN,NaN,NaN,Mount Pleasant,1.0,NaN,11,2,Fraud
1,2010-04,2009-01-01 12:00:00,Y,,,1,121000,9,EMBEZZLEMENT-VALUE OVER $150,,...,NaN,NaN,NaN,NaN,Midtown,9.0,NaN,12,2,Embezzlement
2,2010-04,2009-01-01 12:00:00,Y,,,1,121000,9,EMBEZZLEMENT-VALUE OVER $150,,...,NaN,NaN,NaN,NaN,Midtown,9.0,NaN,12,2,Embezzlement
3,2010-04,2010-01-01 00:01:00,Y,,,1,021000,2,RAPE -- FORCIBLE,5900,...,NaN,NaN,NaN,NaN,Princeton Heights,1.0,2.0,02,1,Forcible Rape
4,2010-04,2010-01-01 12:00:00,Y,,,1,052312,8,BURGLARY-BUSINESS/UNK TIME/FORC ENT/UNOCCUPIED,4902,...,NaN,NaN,NaN,NaN,Kingsway East,8.0,NaN,05,1,Burglary


In [9]:
# Fix some missing values

for c in ('flag_admin', 'flag_crime', 'flag_cleanup','flag_unfounded'):
    df[c] = df[c].fillna('N').replace(' ', 'N')

for c in ('location_name', 'location_comment'):
    df[c] = df[c].replace(' ', np.nan)
    

In [10]:
# There are a lot of earlier crimes, which were discovered after the data collection
# started, but ocurred much eariler. Removing these because those years will not be complete samples. 
# 2008 seems to be the firs full year. 
df = df[df.date_occured.dt.year >= 2008]

In [11]:
# Convert the geo positions from state plane to WGS84
import geopandas  as gpd
df['geometry'] = gpd.points_from_xy(df.x, df.y) #, crs = 26996) # East MO state plane
gdf = gpd.GeoDataFrame(df, crs = 26996).to_crs(4326)

In [12]:
# Done!
gdf.to_csv('st_louis_crime.csv')